In [ ]:
import pandas as pd
import os

# Read and merge CSV files into big one

In [ ]:
folder_path = "raw_data"

In [4]:
# Show list of CSV files that will be merged
csv2merge = []
for filename in os.listdir(folder_path):
    if filename.endswith(".csv") and not filename.startswith("test_"): # set conditions to filter files
        csv2merge.append(filename)
[(n, f) for n, f in enumerate(csv2merge)]

[(0, 'att.csv'),
 (1, 'gps.csv'),
 (2, 'Ilb1.csv'),
 (3, 'Ilb2.csv'),
 (4, 'Ilb3.csv'),
 (5, 'Ilb4.csv'),
 (6, 'Ilb5.csv'),
 (7, 'Ilb6.csv'),
 (8, 'Ilb7.csv'),
 (9, 'Ilb8.csv'),
 (10, 'pos.csv')]

In [28]:
# Open random CSV file from list for exploring (is it correct separator, encoding, etc.)
df = pd.read_csv(os.path.join(folder_path, csv2merge[2]), encoding='utf-8', delimiter=',', header=0)
df.head()

,timestamp,TimeUS,Heading,Pitch,Roll,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,1.749653e+09,73844917,207.889999,6.49,-3.10,0.75748,0.01481,-0.13135,0.113984,0.052751,-0.991858
1,1.749653e+09,73854241,207.889999,6.49,-3.09,0.79922,0.01075,-0.13142,0.113707,0.051803,-0.992360
2,1.749653e+09,73864746,207.889999,6.49,-3.08,0.70018,-0.04003,-0.06693,0.111788,0.051101,-0.992464
3,1.749653e+09,73874859,207.889999,6.49,-3.08,0.79071,-0.02418,-0.06328,0.110342,0.050614,-0.992118
4,1.749653e+09,73884128,207.889999,6.49,-3.07,0.73019,0.04813,-0.05778,0.112285,0.050794,-0.991981


## Explore descriptive statistics of separate data set

In [29]:
df.shape

(134667, 11)

In [7]:
# Custom functions for aggregation

# Function to calculate max to min ratio
def max_min_ratio(x):
    return x.max() / x.min() if x.min() != 0 else None



In [30]:
# Aggregation of summary statistics (replace unnecessary columns)
df.loc[:,'Heading':].agg(
    ['count', 'sum', 'mean', 'std', 'min', 'max', max_min_ratio]) \
    .T.style.format('{:.1f}')

,count,sum,mean,std,min,max,max_min_ratio
Heading,134667.0,27467111.2,204.0,63.4,0.0,360.0,nan
Pitch,134667.0,490145.5,3.6,4.2,-11.9,20.4,-1.7
Roll,134667.0,-281328.5,-2.1,15.6,-66.4,61.8,-0.9
GyrX,134667.0,-1969.4,-0.0,19.0,-120.0,145.7,-1.2
GyrY,134667.0,338094.3,2.5,7.1,-49.1,56.3,-1.1
GyrZ,134667.0,162372.4,1.2,11.2,-73.1,59.4,-0.8
AccX,134667.0,10884.9,0.1,0.1,-1.0,1.0,-1.0
AccY,134667.0,9468.9,0.1,0.1,-1.2,1.3,-1.0
AccZ,134667.0,-139907.0,-1.0,0.2,-3.1,0.4,-0.1


In [31]:
df.loc[:,'Heading':].corr().T.style.format('{:.2f}').background_gradient(cmap='cividis') # 'viridis', 'plasma', 'inferno', 'magma', 'cividis'

,Heading,Pitch,Roll,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
Heading,1.00,-0.07,-0.01,-0.01,-0.15,-0.03,-0.05,-0.03,0.07
Pitch,-0.07,1.00,-0.05,0.05,-0.07,-0.09,0.39,-0.01,-0.02
Roll,-0.01,-0.05,1.00,-0.02,0.00,0.75,0.01,0.04,-0.03
GyrX,-0.01,0.05,-0.02,1.00,0.01,0.01,0.00,0.03,-0.05
GyrY,-0.15,-0.07,0.00,0.01,1.00,-0.02,0.10,0.15,-0.51
GyrZ,-0.03,-0.09,0.75,0.01,-0.02,1.00,-0.07,0.21,0.03
AccX,-0.05,0.39,0.01,0.00,0.10,-0.07,1.00,-0.13,-0.33
AccY,-0.03,-0.01,0.04,0.03,0.15,0.21,-0.13,1.00,-0.09
AccZ,0.07,-0.02,-0.03,-0.05,-0.51,0.03,-0.33,-0.09,1.00


In [ ]:
# Create a copy of the df DataFrame if we need to process it further
#df_gps = df.copy()
#df_pos = df.copy()

## Look at differencies in timestamp for selected file

In [18]:
# RENAME df_gps TO APRICIATE NAME FOR SAVING NEW DATAFRAME
df_buf = pd.DataFrame()
df_buf['timestamp, sec'] = df.loc[:,['timestamp']]   
df_buf['timestamp_diff, sec'] = df.loc[:,['timestamp']].diff()
#df_gps['Data Rate, Hz'] = df_gps['timestamp_diff'] * 100

In [19]:
df_buf.iloc[1:10].style.format('{:.4f}')

,"timestamp, sec","timestamp_diff, sec"
1,1749652624.1266,0.0088
2,1749652624.1372,0.0105
3,1749652624.1473,0.0101
4,1749652624.1565,0.0093
5,1749652624.1666,0.0100
6,1749652624.1770,0.0104
7,1749652624.1876,0.0106
8,1749652624.1971,0.0095
9,1749652624.2073,0.0102


In [21]:
df_buf['timestamp_diff, sec'] \
    .aggregate(min = 'min',
               max = 'max',
               mean = 'mean',
               max_min = lambda x: x.max() / x.min()) \
    .map('{:.2f}'.format)

min           0.00
max           0.20
mean          0.01
max_min    2747.28
Name: timestamp_diff, sec, dtype: object

In [ ]:
# Clear df
del df

## Read and merged files

In [11]:
# Set path to start file whick will be a reference for merging
start_file = "Ilb1.csv"
df0 = pd.read_csv(os.path.join(folder_path, start_file), encoding='utf-8', delimiter=',', header=0)
df_merged = df0.copy()
threshold = 0.01 # Set threshold for merging, in seconds

In [12]:
# Check CSV files that will be merged
print("Processing files:")
for filename in os.listdir(folder_path):
    if (filename != start_file) and filename.startswith("Ilb"): # set conditions to filter files
        print(filename)

Processing files:
Ilb2.csv
Ilb3.csv
Ilb4.csv
Ilb5.csv
Ilb6.csv
Ilb7.csv
Ilb8.csv


In [13]:
# Read CSV files and merge them into a single DataFrame
for filename in os.listdir(folder_path):
    if (filename != start_file) and filename.startswith("Ilb"): # set conditions to filter files
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, encoding='utf-8', delimiter=',', header=0)
        df.drop(columns = ['TimeUS'], inplace=True) # Not needed dublicate column
        df_merged = pd.merge_asof(df_merged, df, on='timestamp', tolerance=threshold, direction='nearest', allow_exact_matches=True)

## Get summary statistics of merged data and save result

In [14]:
print(f"List of columns:\n {df_merged.columns}")

List of columns:
 Index(['timestamp', 'TimeUS', 'Heading', 'Pitch', 'Roll', 'GyrX', 'GyrY',
       'GyrZ', 'AccX', 'AccY', 'AccZ', 'MagX', 'MagY', 'MagZ', 'USW', 'Vinp',
       'Temp', 'Lat', 'Lng', 'Alt', 'ESpd', 'NSpd', 'VSpd', 'LatGnss',
       'LngGnss', 'AltGnss', 'HSpdGnss', 'TrackGnss', 'VSpdGnss', 'msgps',
       'GNSSinf', 'GNSSinf2', 'nSVs', 'LatencyPos', 'LatencyVel',
       'AnglePosType', 'HeadingGnss', 'LatencymsHead', 'Pbar', 'Hbar',
       'NewGps', 'Resv', 'AirSpd', 'NWind', 'EWind', 'NWindStd', 'EWindStd',
       'LatExt', 'LngExt', 'AltExt', 'LatExtStd', 'LngExtStd', 'AltExtStd',
       'ExtPosLatency', 'LocLat', 'LocLon', 'LocAlt', 'Dop', 'DopStd',
       'NewAidD'],
      dtype='object')


In [15]:
df_merged.head(10)

,timestamp,TimeUS,Heading,Pitch,Roll,GyrX,GyrY,GyrZ,AccX,AccY,...,LatExtStd,LngExtStd,AltExtStd,ExtPosLatency,LocLat,LocLon,LocAlt,Dop,DopStd,NewAidD
0,1.749653e+09,73844917,207.889999,6.49,-3.10,0.75748,0.01481,-0.13135,0.113984,0.052751,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.749653e+09,73854241,207.889999,6.49,-3.09,0.79922,0.01075,-0.13142,0.113707,0.051803,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.749653e+09,73864746,207.889999,6.49,-3.08,0.70018,-0.04003,-0.06693,0.111788,0.051101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.749653e+09,73874859,207.889999,6.49,-3.08,0.79071,-0.02418,-0.06328,0.110342,0.050614,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.749653e+09,73884128,207.889999,6.49,-3.07,0.73019,0.04813,-0.05778,0.112285,0.050794,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,1.749653e+09,73894175,207.889999,6.49,-3.06,0.62104,0.05523,-0.02896,0.112541,0.050806,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,1.749653e+09,73904548,207.889999,6.49,-3.06,0.48444,0.08091,-0.00747,0.111923,0.050300,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,1.749653e+09,73915165,207.940002,6.49,-3.05,0.34032,0.08610,-0.03545,0.112857,0.050032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,1.749653e+09,73924712,207.940002,6.49,-3.05,0.24344,-0.04227,-0.03029,0.113310,0.048481,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,1.749653e+09,73934868,207.940002,6.49,-3.05,0.19767,-0.01540,-0.03970,0.113749,0.048343,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [16]:
df_merged.shape

(134667, 60)

In [17]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134667 entries, 0 to 134666
Data columns (total 60 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   timestamp      134667 non-null  float64
 1   TimeUS         134667 non-null  int64  
 2   Heading        134667 non-null  float64
 3   Pitch          134667 non-null  float64
 4   Roll           134667 non-null  float64
 5   GyrX           134667 non-null  float64
 6   GyrY           134667 non-null  float64
 7   GyrZ           134667 non-null  float64
 8   AccX           134667 non-null  float64
 9   AccY           134667 non-null  float64
 10  AccZ           134667 non-null  float64
 11  MagX           134667 non-null  float64
 12  MagY           134667 non-null  float64
 13  MagZ           134667 non-null  float64
 14  USW            134667 non-null  int64  
 15  Vinp           134667 non-null  float64
 16  Temp           134667 non-null  float64
 17  Lat            134667 non-nul

In [24]:
# Aggregation of summary statistics (replace unnecessary columns)
df_merged.loc[:,'Heading':].agg(
    ['count', 'sum', 'mean', 'std', 'min', 'max', max_min_ratio]) \
    .T.style.format('{:.1f}')

,count,sum,mean,std,min,max,max_min_ratio
Heading,134667.0,27467111.2,204.0,63.4,0.0,360.0,nan
Pitch,134667.0,490145.5,3.6,4.2,-11.9,20.4,-1.7
Roll,134667.0,-281328.5,-2.1,15.6,-66.4,61.8,-0.9
GyrX,134667.0,-1969.4,-0.0,19.0,-120.0,145.7,-1.2
GyrY,134667.0,338094.3,2.5,7.1,-49.1,56.3,-1.1
GyrZ,134667.0,162372.4,1.2,11.2,-73.1,59.4,-0.8
AccX,134667.0,10884.9,0.1,0.1,-1.0,1.0,-1.0
AccY,134667.0,9468.9,0.1,0.1,-1.2,1.3,-1.0
AccZ,134667.0,-139907.0,-1.0,0.2,-3.1,0.4,-0.1
MagX,134667.0,-1748636810.0,-12984.9,17022.4,-39300.0,51690.0,-1.3


In [ ]:
# Exclude colums with zeros
df_merged.loc[:, (df_merged != 0).any(axis=0)].corr().T.style.format('{:.2f}').background_gradient(cmap='cividis') 
#df_merged_stat = df_merged.loc[:, (df_merged != 0).any(axis=0)].dropna(axis=1, how='all') # Exclude columns with all NaN values
#df_merged_stat.corr().T.style.format('{:.2f}').background_gradient(cmap='cividis') # 'viridis', 'plasma', 'inferno', 'magma', 'cividis'

c:\Users\Engineer\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\io\formats\style.py:3807: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
c:\Users\Engineer\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\io\formats\style.py:3808: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,timestamp,TimeUS,Heading,Pitch,Roll,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,MagX,MagY,MagZ,Vinp,Temp,Lat,Lng,Alt,ESpd,NSpd,VSpd,LatGnss,LngGnss,AltGnss,HSpdGnss,TrackGnss,VSpdGnss,msgps,GNSSinf,GNSSinf2,nSVs,LatencyPos,LatencyVel,Pbar,Hbar,NewGps,AirSpd
timestamp,1.00,1.00,-0.04,-0.31,-0.26,0.01,0.16,-0.17,-0.26,0.05,-0.08,0.08,-0.31,0.44,-0.11,-0.30,0.27,-0.64,0.57,0.01,0.00,-0.04,-0.22,-0.42,0.55,0.55,0.01,-0.05,0.09,0.80,nan,-0.61,0.17,0.17,-0.64,0.64,0.00,0.69
TimeUS,1.00,1.00,-0.04,-0.31,-0.26,0.01,0.16,-0.17,-0.26,0.05,-0.08,0.08,-0.31,0.44,-0.11,-0.30,0.27,-0.64,0.57,0.01,0.00,-0.04,-0.22,-0.42,0.55,0.55,0.01,-0.05,0.09,0.80,nan,-0.61,0.17,0.17,-0.64,0.64,0.00,0.69
Heading,-0.04,-0.04,1.00,-0.07,-0.01,-0.01,-0.15,-0.03,-0.05,-0.03,0.07,0.04,0.56,0.02,0.02,0.02,-0.01,0.06,-0.15,-0.71,0.03,-0.08,0.03,0.04,-0.14,-0.20,0.66,-0.08,-0.00,-0.05,nan,0.07,-0.01,-0.01,0.14,-0.14,0.00,-0.03
Pitch,-0.31,-0.31,-0.07,1.00,-0.05,0.05,-0.07,-0.09,0.39,-0.01,-0.02,-0.09,-0.06,-0.28,0.03,-0.02,-0.08,0.20,-0.44,0.15,0.11,0.65,0.09,0.25,-0.44,-0.24,-0.06,0.57,-0.03,-0.34,nan,0.23,-0.10,-0.10,0.46,-0.46,0.00,-0.44
Roll,-0.26,-0.26,-0.01,-0.05,1.00,-0.02,0.00,0.75,0.01,0.04,-0.03,0.20,0.61,-0.03,0.04,0.30,0.02,0.13,0.03,0.01,0.17,0.13,0.12,-0.18,0.05,0.13,-0.00,0.15,-0.01,-0.06,nan,0.14,0.02,0.02,-0.02,0.02,-0.00,0.04
GyrX,0.01,0.01,-0.01,0.05,-0.02,1.00,0.01,0.01,0.00,0.03,-0.05,-0.00,-0.01,-0.02,-0.00,-0.01,-0.02,-0.01,-0.01,-0.01,0.00,0.02,-0.03,0.00,-0.01,0.00,-0.02,0.00,0.00,0.00,nan,-0.01,0.00,0.00,0.01,-0.01,-0.00,0.00
GyrY,0.16,0.16,-0.15,-0.07,0.00,0.01,1.00,-0.02,0.10,0.15,-0.51,0.34,-0.17,-0.10,-0.01,0.01,0.02,-0.16,0.14,0.21,0.31,-0.07,-0.09,-0.18,0.14,0.45,-0.10,-0.16,0.02,0.15,nan,-0.14,0.04,0.04,-0.15,0.15,-0.00,0.24
GyrZ,-0.17,-0.17,-0.03,-0.09,0.75,0.01,-0.02,1.00,-0.07,0.21,0.03,0.13,0.44,-0.01,0.03,0.24,0.00,0.09,0.04,0.03,0.08,0.05,0.09,-0.17,0.05,0.09,-0.02,0.11,-0.00,-0.00,nan,0.11,0.03,0.03,-0.03,0.03,-0.00,0.07
AccX,-0.26,-0.26,-0.05,0.39,0.01,0.00,0.10,-0.07,1.00,-0.13,-0.33,0.20,-0.04,-0.14,0.03,0.06,0.04,0.15,-0.32,0.16,0.24,0.16,0.20,0.19,-0.31,-0.05,-0.06,0.12,-0.02,-0.29,nan,0.21,-0.07,-0.07,0.34,-0.34,-0.00,-0.38
AccY,0.05,0.05,-0.03,-0.01,0.04,0.03,0.15,0.21,-0.13,1.00,-0.09,-0.05,0.03,0.05,-0.01,0.04,-0.02,-0.02,0.04,0.04,-0.10,-0.07,-0.03,-0.08,0.05,0.01,-0.04,-0.09,0.01,0.12,nan,0.01,0.03,0.03,-0.05,0.05,-0.00,0.09


## Test how timestamps from different files relative to merged one

In [ ]:
# Get random row in merged DataFrame and from particular file to compare

# Define row indices to compare
a, b = 11, 15

# Get random file
random_file = 'Ilb6.csv'  # Replace with a random file name from the folder
df_rf = pd.read_csv(os.path.join(folder_path, random_file), encoding='utf-8', delimiter=',', header=0)

# How data from the random file looks like
df_rf.loc[a:b,:].style.format({'timestamp': '{:.4f}'}) # Format timestamp for display

In [ ]:
# How data from the random file looks like in the merged DataFrame. 'TimeUS' value is shown from the merged DataFrame
df_merged.loc[a:b, df_rf.columns].style.format({'timestamp': '{:.4f}'})

## Save resuls!

In [ ]:
# Save merged DataFrame to CSV
output_file = "merged_data.csv"
output_folder = "output"
# Create subfolder for storing output file
output_path = os.path.join(folder_path, output_folder)

if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)
df_merged.to_csv(os.path.join(output_path, output_file), index=False, mode='w', encoding='utf-8')